# 1. INTRODUCTION
   Cyclistic, is a fictional bike-share company in Chicago. Cyclistic offers reclining bikes, hand tricycles, and cargo bikes, making bike-share more inclusive to people with disabilities and riders who can’t use a standard two-wheeled bike.
Cyclistic provides flexibility of its pricing plans in order to appeal to the broad consumer segments. Single-ride passes, full-day passes, and annual memberships. Customers who purchase single-ride or full-day passes are referred to as casual riders. Customers who purchase annual memberships are Cyclistic members. The finance analysts have concluded that annual members are much more profitable than casual riders so the head of marketing has decided that maximizing higher annual membership would be more profitable and better for the overall companies’ growth. 
The Business task of this project would be to find ways to make the Cyclistic subscriptions more profitable, the specific task would be to find the difference in patterns and trends between the members and casual riders in order for better targeted marketing that would drive greater membership subscriptions.
In this project, I will be analysing the data for the bike share company found in [this link.](http://www.divvybikes.com/data)


   

# 2. PRELIMINARY WRANGLING
##    2.1 Data Gathering

In [1]:
import pandas as pd
import numpy as np


In [2]:
july_21 = pd.read_csv("202107-divvy-tripdata.csv")
august_21 = pd.read_csv("202108-divvy-tripdata.csv")
september_21 = pd.read_csv("202109-divvy-tripdata.csv")
october_21 = pd.read_csv("202110-divvy-tripdata.csv")
november_21 = pd.read_csv("202111-divvy-tripdata.csv")
december_21 = pd.read_csv("202112-divvy-tripdata.csv")
january_22 = pd.read_csv("202201-divvy-tripdata.csv")
febuary_22 = pd.read_csv("202202-divvy-tripdata.csv")
march_22 = pd.read_csv("202203-divvy-tripdata.csv")
april_22 = pd.read_csv("202204-divvy-tripdata.csv")
may_22 = pd.read_csv("202205-divvy-tripdata.csv")
june_22 = pd.read_csv("202206-divvy-tripdata.csv")

In [3]:
merged_divvy = pd.concat([july_21, august_21, september_21, october_21, november_21, december_21, january_22, febuary_22, march_22, april_22, may_22, june_22], axis=0, ignore_index=True)

 ##  2.2  Getting a concise summary

In [4]:
# Overview of data shape and composition
print(merged_divvy.info())
print(merged_divvy.dtypes)
print(merged_divvy.head(10))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5900385 entries, 0 to 5900384
Data columns (total 13 columns):
 #   Column              Dtype  
---  ------              -----  
 0   ride_id             object 
 1   rideable_type       object 
 2   started_at          object 
 3   ended_at            object 
 4   start_station_name  object 
 5   start_station_id    object 
 6   end_station_name    object 
 7   end_station_id      object 
 8   start_lat           float64
 9   start_lng           float64
 10  end_lat             float64
 11  end_lng             float64
 12  member_casual       object 
dtypes: float64(4), object(9)
memory usage: 585.2+ MB
None
ride_id                object
rideable_type          object
started_at             object
ended_at               object
start_station_name     object
start_station_id       object
end_station_name       object
end_station_id         object
start_lat             float64
start_lng             float64
end_lat               float64
end

In [5]:
# checking the column names
merged_divvy.columns

Index(['ride_id', 'rideable_type', 'started_at', 'ended_at',
       'start_station_name', 'start_station_id', 'end_station_name',
       'end_station_id', 'start_lat', 'start_lng', 'end_lat', 'end_lng',
       'member_casual'],
      dtype='object')

In [6]:
# Checking nulls
merged_divvy.isnull().sum()

ride_id                    0
rideable_type              0
started_at                 0
ended_at                   0
start_station_name    836018
start_station_id      836015
end_station_name      892103
end_station_id        892103
start_lat                  0
start_lng                  0
end_lat                 5374
end_lng                 5374
member_casual              0
dtype: int64

In [7]:
# Checking for duplicates
merged_divvy.duplicated().sum()

0

In [8]:
# Getting count of unique values in categorical columns to check for invalid entries
print(merged_divvy.rideable_type.value_counts())
print(merged_divvy.member_casual.value_counts())
print(merged_divvy.start_station_name.value_counts())

classic_bike     3189377
electric_bike    2457637
docked_bike       253371
Name: rideable_type, dtype: int64
member    3342158
casual    2558227
Name: member_casual, dtype: int64
Streeter Dr & Grand Ave                          82619
Michigan Ave & Oak St                            43633
Wells St & Concord Ln                            43495
DuSable Lake Shore Dr & North Blvd               40515
Millennium Park                                  40322
                                                 ...  
Public Rack - Sawyer Ave & Bryn Mawr Ave - SE        1
Lake Park Ave & 44th St                              1
Evans Ave & 63rd St                                  1
West Chatham Park                                    1
Keef Ave & South Chicago Ave                         1
Name: start_station_name, Length: 1293, dtype: int64


# ASSESSMENT 
#### ISSUE1: 
  The columns ['start_station_name', 'start_station_id', 'end_station_name',
       'end_station_id', 'end_lat', 'end_lng'] have null values which would be removed.
#### ISSUE2: 
   Columns ['started_at', 'ended_at'] needs to be changed into datetime formate
#### ISSUE3: 
    New columns for ['travel_time', 'day_of_week', 'hour', 'month','travel_distance'] need to be created
#### ISSUE4: 
  Finally, subsetting for columns needed for analysis.



### 2.3  Cleaning and manipulsting the data set

In [9]:
# creating a copy of the dataset
divvy_copy = merged_divvy.copy()

divvy_copy.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,0A1B623926EF4E16,docked_bike,2021-07-02 14:44:36,2021-07-02 15:19:58,Michigan Ave & Washington St,13001,Halsted St & North Branch St,KA1504000117,41.883984,-87.624684,41.899368,-87.648480,casual
1,B2D5583A5A5E76EE,classic_bike,2021-07-07 16:57:42,2021-07-07 17:16:09,California Ave & Cortez St,17660,Wood St & Hubbard St,13432,41.900363,-87.696704,41.889899,-87.671473,casual
2,6F264597DDBF427A,classic_bike,2021-07-25 11:30:55,2021-07-25 11:48:45,Wabash Ave & 16th St,SL-012,Rush St & Hubbard St,KA1503000044,41.860384,-87.625813,41.890173,-87.626185,member
3,379B58EAB20E8AA5,classic_bike,2021-07-08 22:08:30,2021-07-08 22:23:32,California Ave & Cortez St,17660,Carpenter St & Huron St,13196,41.900363,-87.696704,41.894556,-87.653449,member
4,6615C1E4EB08E8FB,electric_bike,2021-07-28 16:08:06,2021-07-28 16:27:09,California Ave & Cortez St,17660,Elizabeth (May) St & Fulton St,13197,41.900350,-87.696682,41.886593,-87.658387,casual


ISSUE1: Removing null values

In [10]:
# removing null values
divvy_copy = divvy_copy[(divvy_copy['start_station_name'].notnull()) & (divvy_copy['start_station_id'].notnull()) \
                         & (divvy_copy['end_station_name'].notnull()) & (divvy_copy['end_station_id'].notnull())\
                         & (divvy_copy['end_station_id'].notnull()) & (divvy_copy['end_lat'].notnull())\
                         & (divvy_copy['end_lng'].notnull())]

ISSUE2: Changing the started_at and ended_at columns to date time formats

In [11]:
divvy_copy['started_at'] = pd.to_datetime(divvy_copy['started_at'])
divvy_copy['ended_at'] = pd.to_datetime(divvy_copy['ended_at'])

ISSUE3: Creating new columns

In [12]:
# creating a column for the time each trip takes
travel_time = divvy_copy['ended_at'] - divvy_copy['started_at']
divvy_copy['travel_time'] = travel_time.dt.total_seconds().div(60).astype(int)

In [13]:
divvy_copy['day_of_week'] = divvy_copy['started_at'].dt.day_name()
divvy_copy['hour'] = divvy_copy['started_at'].dt.hour
divvy_copy['month'] = divvy_copy['started_at'].dt.month

print(divvy_copy['day_of_week'].head())
print(divvy_copy['hour'].head())
print(divvy_copy['month'].head())

0       Friday
1    Wednesday
2       Sunday
3     Thursday
4    Wednesday
Name: day_of_week, dtype: object
0    14
1    16
2    11
3    22
4    16
Name: hour, dtype: int64
0    7
1    7
2    7
3    7
4    7
Name: month, dtype: int64


In [14]:
# creating a column for the distance of each trip in km
import h3


divvy_copy['travel_distance'] = divvy_copy.apply(lambda row: h3.point_dist((row['start_lat'], row['start_lng']), (row['end_lat'], row['end_lng'])), axis=1)

In [15]:
divvy_copy.columns

Index(['ride_id', 'rideable_type', 'started_at', 'ended_at',
       'start_station_name', 'start_station_id', 'end_station_name',
       'end_station_id', 'start_lat', 'start_lng', 'end_lat', 'end_lng',
       'member_casual', 'travel_time', 'day_of_week', 'hour', 'month',
       'travel_distance'],
      dtype='object')

In [16]:
# Replace the numbers 1,2... with January, Febuary, ... in the month column
divvy_copy['month'] = divvy_copy['month'].replace([1,2,3,4,5,6,7,8,9,10,11,12],
        ['January','Febuary', 'March','April','May','June','July','August','September','October','November','December'])

In [17]:
print(divvy_copy.describe())

          start_lat     start_lng       end_lat       end_lng   travel_time  \
count  4.678871e+06  4.678871e+06  4.678871e+06  4.678871e+06  4.678871e+06   
mean   4.190267e+01 -8.764412e+01  4.190296e+01 -8.764436e+01  1.877122e+01   
std    4.105772e-02  2.468359e-02  4.119009e-02  2.406638e-02  1.186628e+02   
min    4.164850e+01 -8.783325e+01  4.164850e+01 -8.783000e+01 -1.290000e+02   
25%    4.188200e+01 -8.765694e+01  4.188213e+01 -8.765709e+01  6.000000e+00   
50%    4.189754e+01 -8.764117e+01  4.189776e+01 -8.764117e+01  1.100000e+01   
75%    4.192628e+01 -8.762784e+01  4.192871e+01 -8.762784e+01  2.000000e+01   
max    4.563503e+01 -7.379648e+01  4.216812e+01 -8.752740e+01  4.910700e+04   

               hour  travel_distance  
count  4.678871e+06     4.678871e+06  
mean   1.421040e+01     2.111984e+00  
std    5.005214e+00     1.945089e+00  
min    0.000000e+00     0.000000e+00  
25%    1.100000e+01     8.974806e-01  
50%    1.500000e+01     1.599196e+00  
75%    1.800000

In [18]:
divvy_copy = divvy_copy[divvy_copy['travel_time'] > 0]

In [19]:
print(divvy_copy.describe())

          start_lat     start_lng       end_lat       end_lng   travel_time  \
count  4.612376e+06  4.612376e+06  4.612376e+06  4.612376e+06  4.612376e+06   
mean   4.190272e+01 -8.764409e+01  4.190300e+01 -8.764433e+01  1.904224e+01   
std    4.100764e-02  2.461853e-02  4.114180e-02  2.399085e-02  1.194934e+02   
min    4.164850e+01 -8.783325e+01  4.164850e+01 -8.783000e+01  1.000000e+00   
25%    4.188203e+01 -8.765694e+01  4.188213e+01 -8.765703e+01  6.000000e+00   
50%    4.189759e+01 -8.764116e+01  4.189776e+01 -8.764117e+01  1.100000e+01   
75%    4.192628e+01 -8.762784e+01  4.192871e+01 -8.762784e+01  2.000000e+01   
max    4.563503e+01 -7.379648e+01  4.216812e+01 -8.752740e+01  4.910700e+04   

               hour  travel_distance  
count  4.612376e+06     4.612376e+06  
mean   1.420955e+01     2.142257e+00  
std    5.006082e+00     1.942495e+00  
min    0.000000e+00     0.000000e+00  
25%    1.100000e+01     9.300862e-01  
50%    1.500000e+01     1.617814e+00  
75%    1.800000

Issue4: Creating a subset for only rows to be used in my analysis

In [21]:
#creating a subset for only rows to be used in my analysis
divvy_sub = divvy_copy[['ride_id', 'rideable_type', 'start_station_name', 'end_station_name', 'travel_time', 'day_of_week', 'hour', 'month',
       'travel_distance', 'member_casual']]

In [22]:

divvy_sub.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 4612376 entries, 0 to 5900384
Data columns (total 10 columns):
 #   Column              Dtype   
---  ------              -----   
 0   ride_id             object  
 1   rideable_type       object  
 2   start_station_name  object  
 3   end_station_name    object  
 4   travel_time         int32   
 5   day_of_week         category
 6   hour                category
 7   month               category
 8   travel_distance     float64 
 9   member_casual       object  
dtypes: category(3), float64(1), int32(1), object(5)
memory usage: 277.1+ MB


In [23]:
# Checking nulls
divvy_sub.isnull().sum()

ride_id                   0
rideable_type             0
start_station_name        0
end_station_name          0
travel_time               0
day_of_week               0
hour                  64601
month                     0
travel_distance           0
member_casual             0
dtype: int64

In [24]:
divvy_sub.head(10)

,ride_id,rideable_type,start_station_name,end_station_name,travel_time,day_of_week,hour,month,travel_distance,member_casual
0,0A1B623926EF4E16,docked_bike,Michigan Ave & Washington St,Halsted St & North Branch St,35,Friday,14,July,2.608825,casual
1,B2D5583A5A5E76EE,classic_bike,California Ave & Cortez St,Wood St & Hubbard St,18,Wednesday,16,July,2.390634,casual
2,6F264597DDBF427A,classic_bike,Wabash Ave & 16th St,Rush St & Hubbard St,17,Sunday,11,July,3.312533,member
3,379B58EAB20E8AA5,classic_bike,California Ave & Cortez St,Carpenter St & Huron St,15,Thursday,22,July,3.637853,member
4,6615C1E4EB08E8FB,electric_bike,California Ave & Cortez St,Elizabeth (May) St & Fulton St,19,Wednesday,16,July,3.519575,casual
5,62DC2B32872F9BA8,electric_bike,California Ave & Cortez St,Albany Ave & Bloomingdale Ave,5,Thursday,17,July,1.659964,casual
6,4BBB6E80E6A2A16D,classic_bike,California Ave & Cortez St,Albany Ave & Bloomingdale Ave,11,Wednesday,16,July,1.671583,casual
7,22CA03D32C6BB094,classic_bike,Clark St & North Ave,Clark St & Chicago Ave,8,Saturday,12,July,1.695071,casual
8,61F0D07D1EEE72EE,classic_bike,Sheridan Rd & Montrose Ave,Southport Ave & Clybourn Ave,19,Friday,18,July,4.609252,member
9,09B4551386A8410E,classic_bike,Sheridan Rd & Montrose Ave,Lakefront Trail & Bryn Mawr Ave,13,Thursday,21,July,2.494511,member


In [25]:
#test
print(divvy_sub.day_of_week.value_counts())
print(divvy_sub.month.value_counts())
print(divvy_sub.hour.value_counts())

Saturday     777918
Sunday       684763
Thursday     659884
Friday       642308
Wednesday    624140
Tuesday      619333
Monday       604030
Name: day_of_week, dtype: int64
July         683203
August       666084
September    613365
June         609809
May          494100
October      471464
April        268539
November     252189
March        212906
December     174005
Febuary       87656
January       79056
Name: month, dtype: int64
17    473687
18    401283
16    392141
15    317896
19    291910
14    279621
13    277391
12    274965
11    238165
8     220354
20    203650
10    189911
7     182404
9     174956
21    163479
22    137084
23     97503
6      96911
1      43645
5      36816
2      26717
3      14758
4      12528
24         0
Name: hour, dtype: int64


In [26]:
divvy_sub.to_csv("divvy_clean2.csv")